In [21]:
import pandas as pd
import os

def generate_track_pages(csv_path, output_dir='..\\EDTMProceeding\\Tracks'):
  
    try:
       
        df = pd.read_csv(csv_path)
        
        
        required_columns = ['title', 'abstract', 'authors', 'track', 'invited','transaction']
        if not all(col in df.columns for col in required_columns):
            missing = [col for col in required_columns if col not in df.columns]
            raise ValueError(f"CSV缺少必要列：{missing}")

        os.makedirs(output_dir, exist_ok=True)

        
        track_files = []
        for track_name, group in df.groupby('track'):
            
            safe_name = sanitize_filename(track_name)
            filename = os.path.join(output_dir, f"{safe_name}.html")
            
            
            html_content = generate_single_track_html(track_name, group)
            
           
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(html_content)
            track_files.append((track_name, filename))

        
        generate_index_page(track_files, output_dir)
        
        print(f"Success, at{os.path.abspath(output_dir)}")

    except Exception as e:
        print(f"Fail：{str(e)}")

def sanitize_filename(name):
    
    return "".join([c if c.isalnum() or c in (' ', '_') else '_' for c in name]).strip()

def generate_single_track_html(track_name, df):
    
    papers_html = []
    for _, row in df.iterrows():
        invited_tag = '<span class="invited">[Invited]</span>' if row['invited'] else ''
        papers_html.append(f"""
        <div class="paper">
            <h3 class="title">[<a href={"..\\pdf\\"+row['transaction']+"-final.pdf"}>{row['title']}{invited_tag}</a>]</h3>
            <div class="authors">{row['authors']}</div>
            <div class="abstract">{row['abstract']}</div>
            
        </div>
        """)
    
    return f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>{track_name}</title>
    <style>
        body {{ font-family: Arial, sans-serif; max-width: 800px; margin: 0 auto; padding: 20px; }}
        h1 {{ color: #2c3e50; border-bottom: 2px solid #3498db; }}
        .paper {{ margin: 20px 0; padding: 15px; border-left: 4px solid #3498db; }}
        .title {{ font-weight: bold; color: #2c3e50; }}
        .authors {{ color: #7f8c8d; font-size: 0.9em;font-style: italic; }}
        .abstract {{ color: #666; margin-top: 15px; font-size: 0.95em; }}
        .invited {{ color: #27ae60; font-weight: bold; }}
    </style>
</head>
<body>
    <h1>{track_name} </h1>
    {"".join(papers_html)}
    <p><a href="index.html">← Back</a></p>
</body>
</html>"""

def generate_index_page(track_files, output_dir):
    
    links = []
    for name, path in track_files:
        filename = os.path.basename(path)
        links.append(f'<li><a href="{filename}">{name}</a></li>')
    
    index_html = f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Tracks</title>
    <style>
        body {{ font-family: Arial, sans-serif; max-width: 600px; margin: 0 auto; padding: 20px; }}
        h1 {{ text-align: center; color: #2c3e50; }}
        ul {{ list-style: none; padding: 0; }}
        li {{ margin: 10px 0; padding: 10px; background: #f8f9fa; border-radius: 4px; }}
        a {{ color: #3498db; text-decoration: none; }}
        a:hover {{ text-decoration: underline; }}
    </style>
</head>
<body>
    <h1>EDTM 2025 Papers</h1>
    <ul>{"".join(links)}</ul>
</body>
</html>"""

    with open(os.path.join(output_dir, 'index.html'), 'w', encoding='utf-8') as f:
        f.write(index_html)


if __name__ == "__main__":
    csv_path = "accepted_paper_list.csv"
    generate_track_pages(csv_path)


Success, ate:\The University Of Hong Kong\CANLab - WEN, Bo - WEN, Bo\EDTM\EDTMProceeding\Tracks


Monday
1. 10:30am to 12:30pm
2. 1:40pm to 3:55pm
Tuesday
1. 9:25am to 10:50am
2. 11:00am to 12:30pm
3. 3:00pm to 4:30pm
4. 4:40pm to 6:00pm
Wednesday
1. 9:25am to 10:50am
2. 11:00am to 12:30pm
3. 3:00pm to 5:00pm


In [50]:
import pandas as pd
from pathlib import Path

TRACK_ORDER = [
    "01 Materials",
    "02 Process, Tools, Yield, and Manufacturing",
    "03 Advanced Semiconductor (Logic) Devices",
    "04 Memory Technologies",
    "05 Photonics, Imaging and Display",
    "06 Wide-Bandgap Power and RF Devices",
    "07 Modelling and Simulation",
    "08 Reliability and testing",
    "09 Packaging and Heterogenous Integration",
    "10 Sensor, MEMS, Bio-Electronics",
    "11 Flexible and Wearable Electronics", 
    "12 Nanotechnologies",
    "13 Disruptive Technologies (Metaverse, Neuromorphic Computing, Quantum Computing)",
    '14 Industrial Applications',
    'EDS Special Session'
]

def generate_html_days(csv_path,output_dir='..\\EDTMProceeding\\schedules'):
    os.makedirs(output_dir, exist_ok=True)
    
    df = pd.read_csv(csv_path)
    print(df)
    # df['Day'] = df['day'].map({'March 2025 (Monday)':'Day1','March 2025 (Tuesday)':'Day2','March 2025 (Wednesday)':'Day3'})
    # df['Day'] = df['TimeIndex'].map({0:'Day1',1:'Day1',2:'Day2',3:'Day2',4:'Day3',5:'Day3'})
    
    
    for day in df['day'].unique():
        # ['Day1', 'Day2', 'Day3']:
        day_df = df[df['day'] == day]
        html_content = f'''
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>{day} Papers</title>
    <style>
        body {{ font-family: Arial, sans-serif; max-width: 800px; margin: 0 auto; padding: 20px; }}
        h1 {{ color: #2c3e50; border-bottom: 2px solid #3498db; }}
        .paper {{ margin: 20px 0; padding: 15px; border-left: 4px solid #3498db; }}
        .title {{ font-weight: bold; color: #2c3e50; }}
        .authors {{ color: #7f8c8d; font-size: 0.9em; font-style: italic; }}
        .abstract {{ color: #666; margin-top: 15px; font-size: 0.95em; }}
        .invited {{ color: #27ae60; font-weight: bold; }}
        .venue {{ color: #4C94D8; font-weight: bold; }}
        .time {{color: #BF4E14; font-weight: bold;}}
        .track {{bold;font-weight: bold;}}
        .block-container {{
            padding: 15px;
            
        }}
            .paper-block {{
                    
                    
                    margin: 15px 0;
                    padding: 15px;
                    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                }}
    </style>
</head>
<body>
    <h1>{day}</h1>
'''


        # for track in TRACK_ORDER:
        #     track_df = day_df[day_df['track'] == track]
        #     if not track_df.empty:
        #         html_content += f'<h2>{track}</h2>\n'
        #         html_content += '\n'.join([generate_paper(row) for _, row in track_df.iterrows()])
        
        # for i in range(day_df.__len__()):
        for section in day_df['sections'].unique():
            sec_df = day_df.loc[day_df['sections']==section]
            if not sec_df.empty:
                html_content += '<div class="paper-block">\n'
                html_content += '<details>\n'
                html_content += f'<summary class="time">{section}</summary>\n'
                for track in sec_df['track'].unique():
                    track_df = sec_df[sec_df['track'] == track]
                    if not track_df.empty:
                        # if 
                        location = track_df['venue'].unique()[0]
                        html_content += '<div class="paper-block">\n'
                        html_content += '<details>\n'
                        # html_content += f'<h2>{track}</h2>\n'
                        html_content += f'<summary class="track">{track}</summary>\n'
                        html_content += f'<summary class="venue">{location}</summary>\n'
                        # html_content += f'<summary>{location}</summary>\n'
                        html_content += '\n'.join([generate_paper(row) for _, row in track_df.iterrows()])
                        html_content += '</details>'
                        html_content += '</div>'
                html_content += '</details>'
                html_content += '</div>'

        

        html_content += '''
</body>
</html>'''
        
        
        # with open(f"{day}.html", "w", encoding="utf-8") as f:
        with open(os.path.join(output_dir, f"{day}.html"), 'w', encoding='utf-8') as f:
            f.write(html_content)

def generate_paper(row):

    # invited = '<span class="invited">[Invited]</span> ' if row['invited'] else ''
    if row['invited_keynote']:
        if row['invited_keynote']=='[Invited]':
            invited = '<span class="invited">[Invited]</span> '
        else:
            invited = '<span class="invited">[Keynote]</span> '
    else:
        invited = ''
    
    return f'''
<div class="paper">
    <div class="id_program">{row['time']} {row['Id_program']}</div>
    <div class="title"><a href={"..\\pdf\\"+row['transaction']+"-final.pdf"}>{invited}{row['title']}</a></div>
    <div class="authors">{row['authors_affiliation']}</div>
    <details><summary class='abstract'>Abstract</summary>
    <div class="abstract">{row['abstract']}</div></details>
</div>'''

generate_html_days("program_new.csv")


                                               content  \
0    Impact of Charge Trapping at Defects on the Ro...   
1    Multimode Transistors based on Ion-dynamic Cap...   
2    Understanding the Impact of Hydrogen on the Re...   
3    Investigation of Passivation Layers for Self-a...   
4    Thermal Crosstalk Analysis in Advanced CMOS Ci...   
..                                                 ...   
326  Broadband Vis-NIR Neuromorphic Photodetector B...   
327  Multi-channel Intelligent Electronic Nose for ...   
328  A Multi-Ion Sensing System on a Chip with Edge...   
329  Fully Printed Polymer Gas Sensors Based on Mac...   
330  Dual-Gate Thin-Film Transistor-Based Multi-Par...   

                        day     time       noon  \
0       March 2025 (Monday)  10:30am    morning   
1       March 2025 (Monday)  10:50am    morning   
2       March 2025 (Monday)  11:10am    morning   
3       March 2025 (Monday)  11:30am    morning   
4       March 2025 (Monday)  11:45am    morning 

Works have done 20250227
1. Add tracks and abstract to be folded.
2. Add venue of the tracks.
3. Add time and ID like 'M-1-2-3' in program for each paper.
4. Change the order of papers to list the papers just like what the program does (with several time intervals like 9:25am to 10:50am).
5. Include affiliations of the authors like the program.
To do next if necessary
1. Add 4 Plenary talk
2. Add other information which is not related to papers in platform like Coffee Break or Opening Ceremony or FET 100 Special Session
3. Check if there is mistake or missing (needs help)
4. Add links and abstracts for the 6 pdfs which is missing now
    (1) M-7-1-1 [Keynote] Insulators for Devices Based on 2D Materials. (Keynote) Tibor Grasser, TU Wien, Austria
    Probelm: needs to be accepted but pending now. It is on track 07 (invited).
    (2) M-3-2-1 [Invited] Ferroelectric 3D NAND Storage. Asif Islam Khan, Georgia Institute of Technology, USA
    Problem: same problem as (1), already accepted in platform, will be updated with (1).
    (3) T-9-1-2 [Invited] Advanced Micro-Transfer Printing Technology for Heterogeneous Integration. Yunda Wang, Hong Kong University of Science and Technology, Hong Kong
    Problem: found another paper named "Diode Microheaters for Scalable Actuation in Micro-Transfer Printing" in system which is from the same author in the same invited track 9, seems that they have changed the paper name. checking with the track chair...
    (4) T-1-1-5 [Invited] Phase Change Memory: From Technological Challenges to Materials Science. Zhitang Song, Shanghai Institute of Microsystem and Information Technology, CAS, China
    Problem: missing. The authors will submit a paper for it.
    (5) T-11-2-1 [Keynote] Electronic Fabrics and Wearable Systems: Design, Fabrication, Evaluation. Xiaoming Tao, Hong Kong Polytechnic University, Hong Kong
    Problem: missing. Not found the author/paper in the platform.
    (6)  W-11-3-2 [Invited] AI-boosted wireless near-infrared wearable sensing system for advancing muscle-tracking technology. Wubin Bai, University of North Carolina at Chapel Hill, USA
    Problem: the authors want a different title of the talk from that of the paper named "Magnetic Resonance based Soft Electronic Implant for Wireless Electrotherapy and Thermal Ablation". considering if it needs to add the link or abstract since they look quite different.
